# NER full processing

In [1]:
import polars as pl
import re 
import time
def view_string(long_string, chunk_size=100):     
    return [long_string[i:i+chunk_size] for i in range(0, len(long_string), chunk_size)]

In [35]:
df = pl.read_csv(   
    '/home/sebacastillo/willow/output/news_narcotráfico_related_2023-08-14_1522.csv',
    dtypes={"content_hash": pl.UInt64},     
)

In [36]:
df

date_extract,date_article,topic,content,link,titles,sumaries,authors,portal,state,city,content_hash
str,str,str,str,str,str,str,str,str,str,str,u64
"""2023-08-14 15:…","""2023-07-31T00:…","""narcotráfico""","""Para lograr un…","""https://www.el…","""Ahorra energía…","""Lavar la ropa …","""n-a""","""https://www.lm…","""Río Negro""","""Bariloche""",8469788906430180569
"""2023-08-14 15:…","""2023-08-14T15:…","""narcotráfico""","""Click aquí par…","""https://www.el…","""Visita a empre…","""El secretario …","""n-a""","""https://www.lm…","""Entre Ríos""","""Paraná""",2546624790718344286
"""2023-08-14 15:…","""2023-08-14T09:…","""narcotráfico""","""Dólar blue hoy…","""http://www.cla…","""Previaje 2023:…","""Previaje 2023:…","""Martín Grosz""","""https://www.lm…","""CABA""","""Buenos Aires""",16306887257369882429
"""2023-08-14 15:…","""2023-08-13T00:…","""narcotráfico""","""Estimado lecto…","""https://www.el…","""El cartel del …","""Diario El Día …","""Diario El Dia …","""https://www.lm…","""Buenos Aires""","""La Plata""",13190701768182059490
"""2023-08-14 15:…","""2023-08-11T00:…","""narcotráfico""","""La violencia e…","""https://www.mi…","""Otro crimen en…","""Mientras Peral…","""Lucía Paz Gime…","""https://www.lm…","""Neuquén""","""Plottier""",17359079798055081236
"""2023-08-14 15:…","""2023-08-14T00:…","""narcotráfico""","""27°SAN LUIS - …","""https://www.el…","""Ordenaron 18 a…","""Los mismos efe…","""El Diario De L…","""https://www.lm…","""San Luis""","""San Luis""",13464925544034315656
"""2023-08-14 15:…","""2023-08-12T00:…","""narcotráfico""","""Investigadores…","""https://www.el…","""En una despens…","""Investigadores…","""n-a""","""https://www.lm…","""Misiones""","""Oberá""",12157580530488172524
"""2023-08-14 15:…","""2023-08-12T00:…","""narcotráfico""",""" Director: …","""https://www.an…","""La AFIP incaut…","""Con máquinas r…","""n-a""","""https://www.lm…","""Entre Ríos""","""Paraná""",913931680825709426
"""2023-08-14 15:…","""2023-08-14T15:…","""narcotráfico""","""EN VIVO Hipera…","""https://www.pa…","""""El don del cr…","""La primera obr…","""Https-Demian P…","""https://www.lm…","""Buenos Aires""","""Buenos Aires""",6017811793118880156


# Cleaning febore write to DB

In [4]:
df_clean = (
    df.with_columns([
        pl.col('date_extract').str.strptime(pl.Date, format='%Y-%m-%d %H:%M:%s', strict=True),
        pl.col('date_article').str.slice(0, 10).str.strptime(pl.Date, format='%Y-%m-%d'),
        pl.col("content").str.n_chars().alias("content_nchar"),
        pl.col("content").hash().alias("content_hash"),
        # Add other columns/transformations as required
          # Replace new lines and tabs with a single space, then replace multiple spaces with a single space
        pl.col("content").str.replace_all(r"[\n\t]+", " ")
        .str.replace_all(r"\s{2,}", " ")
        .str.strip().str.strip().alias("content_cleaned"),  # Adjusted
    ])
)


## nuevo match

In [5]:
# excluir falsos positivos
# por extension menor a 400 caracteres: muy poca información
"""df_clean = (
    df_clean
    .filter(pl.col('content_nchar') > 400)
)"""

"df_clean = (\n    df_clean\n    .filter(pl.col('content_nchar') > 400)\n)"

In [6]:
df_clean['titles'].to_list()[:10]

['Ahorra energía y dinero al utilizar tu lavarropa en el momento indicado',
 'Visita a empresarios y organizaciones culturales: Federico Prieto en el departamento Colón',
 'Previaje 2023: cómo averiguar cuánto dinero queda en la tarjeta y hasta qué día se podrá usar',
 'El cartel del medido, tapado por las ramas en 47, 4 y 5',
 'Otro crimen en el conurbano conmociona al país: mataron a un hombre de 50 años en un asalto',
 'Ordenaron 18 allanamientos y detuvieron a tres acusados del crimen de Pérez Algaba',
 'En una despensa de Posadas incautaron 131 dosis de cocaína: dos detenidos con poco más de 7 millones de pesos',
 'La AFIP incautó 60 toneladas de soja valuada en más de 6 millones de pesos',
 '"El don del crimen" de Marco Lucchesi',
 'La agresión a una chica, la disputa de dos jóvenes y una ejecución en una calle de Pocito']

Processin all with loop

In [172]:
index = list(range(1, df_final.shape[0]+1))
articles = df_final['content_cleaned'].to_list()

In [ ]:
import spacy
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline

# Load Spanish stopwords from spacy
nlp = spacy.load("es_core_news_sm")
spanish_stopwords_spacy = spacy.lang.es.stop_words.STOP_WORDS

def remove_stopwords(text):
    """Remove Spanish stopwords from the text."""
    return " ".join([word for word in text.split() if word not in spanish_stopwords_spacy])

def langchain_chunk_text(text):
    
    # Remove stopwords
    # text = remove_stopwords(text)
    
    # Define a basic whitespace tokenizer
    def whitespace_tokenizer(text):
        return text.split()

    custom_text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=60,
        length_function = len,
        #length_function=lambda x: len(whitespace_tokenizer(x))
    )
    
    # Create the chunks using langchain
    documents = custom_text_splitter.create_documents([text])
    
    # Extract the text content from the resulting Document objects
    chunks = [doc.page_content for doc in documents]
    
    return chunks


def ner_on_large_document(text):
    nlp_ner = pipeline(
        "ner",
        model="mrm8488/bert-spanish-cased-finetuned-ner",
        #model="PlanTL-GOB-ES/roberta-base-bne-capitel-ner",
        #tokenizer=('mrm8488/bert-spanish-cased-finetuned-ner', {"use_fast": False}),
        #aggregation_strategy="simple"
        aggregation_strategy="max" # more precision!
        #aggregation_strategy="average"
    )

    chunks = langchain_chunk_text(text)
    all_ner_results = []

    for chunk in chunks:
        ner_results = nlp_ner(chunk)
        all_ner_results.extend(ner_results)

    return all_ner_results

In [ ]:
index = list(range(1, df_final.shape[0]+1))
articles = df_final['content_cleaned'].to_list()

In [174]:
# Initialize an empty DataFrame
df_combined = pl.DataFrame()

for article, i in zip(articles, index):
    # Assume ner_on_large_document is a function that processes the article
    ner = ner_on_large_document(article)
    df = pl.DataFrame(ner)
    df_len = df.shape[0]
    df = df.with_columns(pl.Series("index", [i] * df_len))  
    # Check if the columns of df match those of df_combined or if df_combined is still empty
    if set(df.columns) == set(df_combined.columns) or df_combined.shape[1] == 0:
        # Vertically stack the generated df to the df_combined
        df_combined = pl.concat([df_combined, df])

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.po

In [ ]:
df_combined.write_csv('ner.csv')